# Genetic Algorithm

Genetic Algorithms(GAs) are adaptive heuristic search algorithms that belong to the larger part of evolutionary algorithms. Genetic algorithms are based on the ideas of natural selection and genetics. These are intelligent exploitation of random searches provided with historical data to direct the search into the region of better performance in solution space. `They are commonly used to generate high-quality solutions for optimization problems and search problems.`

Genetic algorithms are based on an analogy with the genetic structure and behavior of chromosomes of the population. Following is the foundation of GAs based on this analogy –  

1. Individuals in the population compete for resources and mate
2. Those individuals who are successful (fittest) then mate to create more offspring than others
3. Genes from the “fittest” parent propagate throughout the generation, that is sometimes parents create offspring which is better than either parent.
4. Thus each successive generation is more suited for their environment.

**Flowchart**
![title](genetic_flowchart.png)

## Python code

In [1]:
import random
import numpy as np

class GeneticAlgorithm:
    def __init__(self,initial_population):
        self.initial_population = initial_population
        self.result = []
    
    def my_func(self,x):
        return x**2 - 16
    
    '''
    This function is used for binary encoding it takes individual(n) as input and convert it into 5 bit binary number,
    where MSB represent its sign(0 for + and 1 for -). It returns the binary number in the form of list 
    for example,
    5 : [0,0,1,0,1]
    -3 : [1,0,0,1,1]
    
    '''
    def get_binary_form(self,n):
        if n not in np.arange(-15,16):
            raise Exception("The initial population should be between 15 to -15")
        x = n
        n = abs(n)

        if n == 0:
            binary = [0,0,0,0,0]
        else:
            for i in range(1, n + 1):
                binary = []
                temp = i
                # Convert decimal number to binary number
                while temp:
                    if temp & 1:
                        binary.insert(0,1)
                    else:
                        binary.insert(0,0)
                    temp >>= 1  # Right shift the bits of temp by 1 position


            if len(binary) != 4:
                for i in range(0, 4- len(binary)):
                    binary.insert(0,0)

            if x < 0:
                binary.insert(0,1)
            else:
                binary.insert(0,0)

        return binary
    
    
    '''
    It takes individuals as input and create its binary form and returns the values in the form of dictionary
    '''
    def generate_binary_dict(self, nums):
        
        
        if (len(nums) < 3):
#           if the population has less than 3 individual this loop appends random integer between -6 and 6
            for i in range(3):
                if len(nums) != 3:
                    nums.append(random.randint(-6, 6))

        bin_dict = {}
        for num in nums:
            bin_dict[num] = self.get_binary_form(num)

        return bin_dict
    
    
    '''
    Only those individual who are fittest can produce next generation, this function helps to find those fittest individuals
    from the given population. For this problem, those individual whose functional value will be closet to zero will be the
    fittest one
    '''
    def fittest(self,nums):
        numbers = {}

        #finding the fuctional value for each individuals
        for num in nums.keys():
            numbers[num] = self.my_func(num)
            # if the answer is found appending it in result
            if numbers[num] == 0:
                self.result.append(num)
    

        # finding the fittest value i.e. the values whose functional value is closet to zero
        sorted_numbers = sorted(numbers.items(), key=lambda item: abs(item[1]))
        
#         sorted_numbers = sorted_numbers[:4]
        fittest_values = dict(sorted_numbers)


        binary_form = self.generate_binary_dict(fittest_values) 

        return binary_form
    
    '''
    it produces offsprings for next generation. This function randomly selects two chromosomes(parents) and produces two 
    offsprings for each combination of chromosome. here, uniform crossover is used
    '''
    def crossover(self, nums):
        keys = list(nums.keys())
        values = list(nums.values())
#         parent1 = values[0]
        toss = [0,1,0,1,0] # 

        offspring1 = []
        offspring2 = []


        for i in range(1,len(values)):
#             parent2 = values[i]
            # selecting the two random parents
            parent1, parent2 = random.sample(values,2)
            o1 = [] # offspring 1 of each parent is store here
            o2 = [] # offspring 2 of each parent is store here
            toss_index = 0
            for p1,p2 in zip(parent1,parent2):
                if toss[toss_index] == 0:
                    o1.append(p1)
                    o2.append(p2)
                elif toss[toss_index] == 1:
                    if p1 == p2:
                        o1.append(p1)
                        o2.append(p2)
                    else:
                        o1.append(p2)
                        o2.append(p1)
                toss_index += 1

            offspring1.append(o1)
            offspring2.append(o2)


        return offspring1, offspring2

    '''
    it used to alter the gene of offspring i.e. flipping any binary bit
    '''
    def mutation(self,of1,of2):
        # each offspring is selected one by one and where mp is 1 than its corresponding bit of offspring is changed 
        # if we have of = [0,0,1,1,0] and mp = [1,,0,1,0,0] then mutated offspring will be
        # of = [1,0,0,1,0]
        mp = [1,0,1,0,0]

        final_offspring = []

        # altering the bits of offspring 1
        for of in of1:
            mutated_offspring = []
            mp_index = 0
            for gene in of:
                if mp[mp_index] == 0:
                    mutated_offspring.append(gene)
                else:
                    if gene == 0:
                        mutated_offspring.append(1)
                    elif gene == 1:
                        mutated_offspring.append(0)

                mp_index += 1

            final_offspring.append(mutated_offspring)

        # altering the bits of offspring 2
        for of in of2:
            mutated_offspring = []
            mp_index = 0
            for gene in of:
                if mp[mp_index] == 0:
                    mutated_offspring.append(gene)
                else:
                    if gene == 0:
                        mutated_offspring.append(1)
                    elif gene == 1:
                        mutated_offspring.append(0)

                mp_index += 1

            final_offspring.append(mutated_offspring)


        return final_offspring
    
    '''
    it converts the final offspring which is in binary form into decimal form which becomes the individuals for next
    generation.
    '''
    def binary_to_decimal(self,offsprings):
        result = []
        for of in offsprings:
            add = 0
            power = 0
            for i in range(1,len(of)):
                add = add + (of[-i]*pow(2,power))
                power += 1

            if of[0] == 1:
                add = -add
            result.append(add)

    #     removing the duplicate elments
        result = list(set(result))
        return result
    
    '''
    It executes all of the above process and print the final offspring from each generation
    '''
    def find(self):
        initial_value = self.initial_population
        for i in range(10):
            initial_value = initial_value
            bin_dict = self.generate_binary_dict(initial_value)
            fittest_chromosomes = self.fittest(bin_dict)
            o1,o2 = self.crossover(fittest_chromosomes)
            next_generation = self.mutation(o1,o2)
            initial_value = self.binary_to_decimal(next_generation)
            print(f"{i + 1} generation: {initial_value}")
            
        
        self.result = list(set(self.result))
        print(f"The roots are: {self.result}")


In [4]:
# initial_population = [-3,-6,1,2,3,5,6,-1,-2]
initial_population = [5,6,7,8,9,10,11,-15,15,14]

ga = GeneticAlgorithm(initial_population=initial_population)
ga.find()

1 generation: [0, 1, 3, 9, 11, -15, -14, -12, -11, -9, -8, -7, -3]
2 generation: [0, 2, 4, 8, 10, 11, 13, 15, -15, -14, -13, -12, -7, -5]
3 generation: [0, 8, 9, 10, 11, -14, -13, -12, -11, -9, -8, -4, -1]
4 generation: [4, 5, 8, 10, 12, 13, 14, 15, -15, -14, -13]
5 generation: [0, 1, 9, 10, 11, -12, -11, -10, -9, -8, -4]
6 generation: [4, 8, 10, 13, 14, 15, -15, -14, -13, -5, -4]
7 generation: [8, 9, 10, 11, -11, -10, -9, -8, -4, -1]
8 generation: [8, 12, 13, 14, 15, -15, -14, -13, -5]
9 generation: [9, 11, -14, -12, -11, -10, -9]
10 generation: [8, 10, 12, 14, 15, -15]
The roots are: [4, -4]


###  Algorithm

**Start:**
This is the initiation point of the algorithm. It's where the process begins and any initial parameters or settings are loaded.


**Creation of initial population:**
In this step, a set of potential solutions (called chromosomes or individuals) is randomly generated. Each chromosome represents a possible solution to the problem being solved. The population size can vary depending on the complexity of the problem.



**Fitness assessment:**
Here, each chromosome in the population is evaluated based on how well it solves the problem. A fitness function is used to assign a fitness score to each chromosome. This score determines how "good" the solution is.



**End condition:**
This is a decision point where the algorithm checks if the termination criteria have been met. These criteria could be:

- A satisfactory solution has been found
- A maximum number of generations has been reached
- The population has converged (solutions are not improving significantly)


**Selection:**
If the end condition is not met, the algorithm moves to selection. Here, chromosomes are chosen to be parents for the next generation. Typically, chromosomes with higher fitness scores have a higher chance of being selected.


**Crossover:**
In this step, genetic information from the selected parents is combined to create new offspring solutions. This mimics biological reproduction and allows the algorithm to explore new potential solutions.



**Creation of new population:**
The offspring created through crossover (and possibly mutation, though not explicitly shown in this flowchart) form a new generation. This new population replaces the old one and the process loops back to the fitness assessment step.



**Choice of "the best" chromosome:**
If the end condition is met, the algorithm selects the best solution found so far. This is typically the chromosome with the highest fitness score across all generations.



**End:**
The algorithm terminates, outputting the best solution found.